# CAMELS-SPAT Large Sample Experiment Tutorial

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import yaml
from datetime import datetime

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Set up plotting style
plt.style.use('default')
%matplotlib inline

In [ ]:
# Configuration for the large sample experiment
experiment_config = {
    'dataset': 'camels-spat',
    'max_watersheds': 5,
    'dry_run': False,
    'experiment_name': 'camelsspat_tutorial',
    'template_config': '/home/darri.eythorsson/code/CONFLUENCE/0_config_files/config_distributed_basin_template.yaml',
    'config_dir': '/home/darri.eythorsson/code/CONFLUENCE/0_config_files/camels_spat',
    'camelsspat_script': '/home/darri.eythorsson/code/CONFLUENCE/9_scripts/run_watersheds_camelsspat.py',
    'camelsspat_dir': '/work/comphyd_lab/data/_to-be-moved/camels-spat-upload/shapefiles/meso-scale/shapes-distributed',
    'metadata_csv': 'camels-spat-metadata.csv'
}

# Create experiment directory
experiment_dir = Path(f"./experiments/{experiment_config['experiment_name']}")
experiment_dir.mkdir(parents=True, exist_ok=True)

# Save configuration
with open(experiment_dir / 'experiment_config.yaml', 'w') as f:
    yaml.dump(experiment_config, f)

In [ ]:
# Import function from the CAMELS-SPAT script to extract shapefile info
sys.path.append(str(Path(experiment_config['camelsspat_script']).parent))
from run_watersheds_camelsspat import extract_shapefile_info

# Check if we already have watershed info cached
watersheds_csv = experiment_dir / 'camelsspat_watersheds.csv'

if watersheds_csv.exists():
    print(f"Loading existing watershed information")
    watersheds_df = pd.read_csv(watersheds_csv)
else:
    print(f"Extracting watershed information...")
    watersheds_df = extract_shapefile_info(experiment_config['camelsspat_dir'])
    watersheds_df.to_csv(watersheds_csv, index=False)

print(f"Found {len(watersheds_df)} watersheds")

In [ ]:
# Load CAMELS-SPAT metadata if available
metadata_path = experiment_config['metadata_csv']

if os.path.exists(metadata_path):
    print(f"Loading and merging metadata")
    metadata_df = pd.read_csv(metadata_path)
    metadata_df.columns = [col.strip() for col in metadata_df.columns]
    
    watersheds_df['Metadata_ID'] = watersheds_df['ID'].str.replace(r'^[A-Z]+_', '', regex=True)
    watersheds_merged = pd.merge(
        watersheds_df, metadata_df, 
        left_on='Metadata_ID', right_on='ID',
        how='left', suffixes=('', '_metadata')
    )
    watersheds_df = watersheds_merged
else:
    print(f"No metadata file found. Proceeding with shapefile information only.")

In [ ]:
'''
# Launch the large sample experiment
cmd = ['python', experiment_config['camelsspat_script']]

# Add command-line arguments
if experiment_config['dry_run']:
    cmd.extend(['--dry-run'])

if experiment_config['max_watersheds'] > 0:
    cmd.extend(['--max-watersheds', str(experiment_config['max_watersheds'])])

print(f"Launching CONFLUENCE for {experiment_config['max_watersheds']} watersheds")
result = subprocess.run(cmd, capture_output=True, text=True)

# Save submission log
with open(experiment_dir / 'submission.log', 'w') as f:
    f.write(result.stdout)
'''

In [ ]:
'''
# Parse submission log and check job status
submission_log = experiment_dir / 'submission.log'
submitted_jobs = []

if submission_log.exists():
    with open(submission_log, 'r') as f:
        log_content = f.read()
    
    # Extract job submissions
    import re
    pattern = r'Domain: ([^,]+), Job ID: (\d+)'
    matches = re.findall(pattern, log_content)
    for domain, job_id in matches:
        submitted_jobs.append({'domain': domain, 'job_id': job_id})
    
    if submitted_jobs:
        jobs_df = pd.DataFrame(submitted_jobs)
        print(f"Submitted {len(jobs_df)} jobs")

# Check job status
def check_job_status(user=None):
    user = user or os.environ.get('USER')
    cmd = ['squeue', '-u', user]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout

print("\nCurrent jobs:")
print(check_job_status())
'''

In [ ]:
# Find completed watershed simulations
confluence_data_dir = Path("/work/comphyd_lab/data/CONFLUENCE_data")
camelsspat_dir = confluence_data_dir / "camels_spat"

completed = []
if camelsspat_dir.exists():
    for domain_dir in camelsspat_dir.glob("domain_*"):
        watershed_id = domain_dir.name.replace('domain_', '')
        sim_dir = domain_dir / "simulations"
        
        if sim_dir.exists() and list(sim_dir.rglob("*.nc")):
            completed.append({
                'watershed_id': watershed_id,
                'domain_dir': domain_dir,
                'sim_dir': sim_dir
            })

print(f"Completed simulations: {len(completed)}")

In [ ]:
# Define functions to load model outputs
def load_summa_output(sim_dir, variable='scalarSWE'):
    import xarray as xr
    summa_output_dir = sim_dir / 'run_1' / 'SUMMA'
    if not summa_output_dir.exists():
        return None
    output_files = list(summa_output_dir.glob("*timestep*.nc"))
    if output_files:
        ds = xr.open_dataset(output_files[0])
        if variable in ds.variables:
            return pd.DataFrame({
                'time': pd.to_datetime(ds.time.values),
                'value': ds[variable].values.flatten()
            })
    return None

def load_streamflow(sim_dir):
    import xarray as xr
    mizuroute_dir = sim_dir / 'run_1' / 'mizuRoute'
    if not mizuroute_dir.exists():
        print(f"mizuRoute directory not found: {mizuroute_dir}")
        return None
    
    output_files = list(mizuroute_dir.glob("*.nc"))
    if not output_files:
        print(f"No netCDF files found in: {mizuroute_dir}")
        return None
    
    try:
        ds = xr.open_dataset(output_files[0])
        
        # Try to find streamflow variable
        for var in ['IRFroutedRunoff', 'routedRunoff', 'discharge']:
            if var in ds.variables:
                # Check dimensions
                var_dims = ds[var].dims
                
                if len(var_dims) == 1 and 'time' in var_dims:
                    # Single dimension (time only)
                    return pd.DataFrame({
                        'time': pd.to_datetime(ds.time.values),
                        'simulated': ds[var].values
                    })
                
                elif len(var_dims) > 1:
                    # Multiple dimensions (likely time and segments/reaches)
                    time_dim = 'time'
                    reach_dims = [d for d in var_dims if d != time_dim]
                    
                    if not reach_dims:
                        print(f"Unexpected dimensions in {var}: {var_dims}")
                        continue
                    
                    reach_dim = reach_dims[0]
                    
                    # Find the right reach/segment to use
                    # If there's a single reach, use it
                    if ds[reach_dim].size == 1:
                        reach_idx = 0
                    else:
                        # TODO: Add logic to find the appropriate outlet reach
                        # For now, use the last reach which is often the outlet
                        reach_idx = ds[reach_dim].size - 1
                    
                    # Extract data for the selected reach
                    flow_data = ds[var].isel({reach_dim: reach_idx}).values
                    
                    # Make sure lengths match
                    if len(ds.time) == len(flow_data):
                        return pd.DataFrame({
                            'time': pd.to_datetime(ds.time.values),
                            'simulated': flow_data
                        })
                    else:
                        print(f"Length mismatch: time ({len(ds.time)}) vs flow ({len(flow_data)})")
        
        print(f"No suitable flow variable found in {output_files[0]}")
        return None
        
    except Exception as e:
        print(f"Error loading streamflow data: {e}")
        return None

def load_observed_streamflow(domain_dir, watershed_id):
    obs_dir = domain_dir / 'observations' / 'streamflow' / 'preprocessed'
    obs_file = list(obs_dir.glob(f"*{watershed_id}*streamflow*.csv"))
    
    if obs_file:
        obs_df = pd.read_csv(obs_file[0])
        time_col = 'datetime' if 'datetime' in obs_df.columns else 'date'
        flow_col = 'discharge_cms' if 'discharge_cms' in obs_df.columns else 'streamflow'
        
        return pd.DataFrame({
            'time': pd.to_datetime(obs_df[time_col]),
            'observed': obs_df[flow_col]
        })
    return None

In [ ]:
# Calculate performance metrics
def calculate_metrics(obs, sim):
    mask = ~(np.isnan(obs) | np.isnan(sim))
    obs, sim = obs[mask], sim[mask]
    
    if len(obs) == 0:
        return {}
    
    # Nash-Sutcliffe Efficiency
    nse = 1 - np.sum((obs - sim)**2) / np.sum((obs - np.mean(obs))**2)
    # RMSE
    rmse = np.sqrt(np.mean((obs - sim)**2))
    # Percent Bias
    pbias = 100 * np.sum(sim - obs) / np.sum(obs) if np.sum(obs) != 0 else np.nan
    # Correlation
    corr = np.corrcoef(obs, sim)[0, 1] if len(obs) > 1 else np.nan
    
    return {'NSE': nse, 'RMSE': rmse, 'PBIAS': pbias, 'Correlation': corr}

# Calculate metrics for completed watersheds
metrics_list = []
for ws in completed:
    sim_flow = load_streamflow(ws['sim_dir'])
    obs_flow = load_observed_streamflow(ws['domain_dir'], ws['watershed_id'])
    
    if sim_flow is not None and obs_flow is not None:
        merged = pd.merge(obs_flow, sim_flow, on='time', how='inner')
        metrics = calculate_metrics(merged['observed'].values, merged['simulated'].values)
        metrics['watershed_id'] = ws['watershed_id']
        metrics_list.append(metrics)

if metrics_list:
    metrics_df = pd.DataFrame(metrics_list)
    print("Performance Metrics:")
    print(metrics_df)
    metrics_df.to_csv(experiment_dir / 'performance_metrics.csv', index=False)

In [ ]:
# Create summary report
if completed:
    print("### CAMELS-SPAT Experiment Summary Report ###")
    print(f"Experiment: {experiment_config['experiment_name']}")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d')}")
    print(f"Completed simulations: {len(completed)}")
    
    if metrics_list:
        print("\nOverall Model Performance:")
        for metric in ['NSE', 'RMSE', 'PBIAS', 'Correlation']:
            if metric in metrics_df.columns:
                print(f"  Average {metric}: {metrics_df[metric].mean():.3f}")
                
    # Save report to file
    report_path = experiment_dir / 'experiment_report.txt'
    with open(report_path, 'w') as f:
        f.write(f"CAMELS-SPAT Experiment Summary\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d')}\n")
        f.write(f"Experiment: {experiment_config['experiment_name']}\n")
    
    print(f"\nSummary report saved to {report_path}")
else:
    print("No completed simulations found yet.")